### [Optimizing Trading Strategy with Zero Lag Moving Average and Fisher Transform](https://medium.com/@kridtapon/optimizing-your-trading-strategy-with-zero-lag-moving-average-and-fisher-transform-64a28c303988)

> Building a Complete Trading System to Enhance Market Performance

The development of a consistently profitable trading strategy necessitates the identification of reliable technical indicators, along with the systematic optimization of their parameters over time. In the present study, a comprehensive trading system is constructed utilizing two lesser-known yet effective indicators: the **Zero Lag Moving Average (ZLMA)** and the **Fisher Transform**.

To evaluate the practical application of these indicators, the system is applied to historical data from a high-performing equity — **_Howmet Aerospace Inc. (HWM)_** — which demonstrated notable market strength over the past year. Prior to the system’s construction and application, a detailed examination of the selected indicators is undertaken to establish a foundation for their usage within the broader trading framework.

#### Understanding the Indicators

1. **Zero Lag Moving Average (ZLMA)**

The ZLMA is an enhanced version of the Exponential Moving Average (EMA) that aims to reduce lag while maintaining smooth trend-following characteristics. It achieves this by applying a double EMA calculation and then adjusting the values to respond more quickly to price changes.

2. **Fisher Transform**

This indicator transforms asset prices into a Gaussian normal distribution, making it easier to identify trend reversals. The Fisher Transform is calculated based on the stock’s highs and lows over a specified period. It also includes a signal line, which is an Exponential Moving Average (EMA) of the Fisher values, helping traders confirm buy and sell signals.

#### Walk-Forward Optimization Approach

- Selecting an initial training period (e.g., 2 years of historical data)
- Identifying optimal indicator parameters for this period by testing multiple ZLMA and Fisher Transform settings combinations.
- Applying the best-performing parameters to the next year’s data for validation.
- Repeating the process each year to adapt to changing market conditions.

#### Backtesting and Performance Evaluation

1. **Downloading historical stock data**

Retrieve stock price data, including open, high, low, close, and volume, from _Yahoo Finance_.

2. **Calculating indicators**

ZLMA and Fisher Transform values are calculated for the training period.

3. **Generating entry and exit signals**

- An **_Entry signal_** is triggered when the stock's price is above the ZLMA and the Fisher Transform is positive.

- An **_Exit signal_** occurs when the stock's price falls below the ZLMA and the Fisher Transform turns negative.

4. **Walk-forward optimization**

The system scans multiple ZLMA and Fisher Transform periods to determine the best-performing combination for each training window. The optimized parameters are then applied to the following year's data.

5. **Backtesting performance**

The strategy is tested using _Vectorbt_, a library that evaluates performance metrics such as total return, drawdowns, and equity curves.

6. **Visualizing results**

The equity curve and trade signals are plotted to analyze performance over time.

In [ ]:
!pip install -q numpy pandas matplotlib yfinance vectorbt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import matplotlib.pyplot as plt
import itertools

# Function to calculate Zero Lag Moving Average (ZLMA)
def calculate_zlma(series, period=20):
    """
    Calculate Zero Lag Moving Average (ZLMA).
    """
    ema1 = series.ewm(span=period, adjust=False).mean()
    ema2 = ema1.ewm(span=period, adjust=False).mean()
    zlma = 2 * ema1 - ema2
    return zlma

# Function to calculate Fisher Transform and its Signal line
def calculate_fisher_transform(df, period=10):
    """
    Calculate Fisher Transform.
    """
    high_rolling = df['High'].rolling(window=period).max()
    low_rolling = df['Low'].rolling(window=period).min()

    # Avoid division by zero
    range_rolling = high_rolling - low_rolling
    range_rolling[range_rolling == 0] = np.nan  # Replace 0 with NaN to avoid errors

    # Calculate X
    X = 2 * ((df['Close'] - low_rolling) / range_rolling - 0.5)

    # Fisher Transform
    fisher = 0.5 * np.log((1 + X) / (1 - X))

    # Signal line (Exponential Moving Average of Fisher)
    fisher_signal = fisher.ewm(span=9, adjust=False).mean()

    return fisher, fisher_signal

# Walk-forward optimization with ZLMA and Fisher Transform
def walk_forward_optimization_zlma_fisher(df, start_year, end_year):
    results = []

    # Define dynamic ranges for ZLMA and Fisher Transform periods
    zlma_period_range = range(1, 101)  # Range for ZLMA periods
    fisher_period_range = range(1, 101)  # Range for Fisher Transform periods

    for test_year in range(start_year + 2, end_year + 1):
        train_start = test_year - 2
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through all combinations of ZLMA and Fisher periods
        for params in itertools.product(zlma_period_range, fisher_period_range):
            zlma_period, fisher_period = params

            # Calculate ZLMA and Fisher Transform indicators on the training data
            train_data['ZLMA'] = calculate_zlma(train_data['Close'], zlma_period)
            train_data['Fisher'], train_data['Fisher_Signal'] = calculate_fisher_transform(train_data, fisher_period)

            # Generate entry and exit signals based on ZLMA and Fisher Transform
            entries = (train_data['Close'] > train_data['ZLMA']) & (train_data['Fisher'] > 0)
            exits = (train_data['Close'] < train_data['ZLMA']) & (train_data['Fisher'] < 0)

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (zlma_period, fisher_period)

        # Test with the best parameters on the test data
        yearly_data = df[(df.index.year >= test_year - 1) & (df.index.year <= test_year)]

        # Apply ZLMA and Fisher Transform indicators
        yearly_data['ZLMA'] = calculate_zlma(yearly_data['Close'], best_params[0])
        yearly_data['Fisher'], yearly_data['Fisher_Signal'] = calculate_fisher_transform(yearly_data, best_params[1])

        # Keep only the second year to avoid missing values from indicator calculation
        yearly_data = yearly_data[yearly_data.index.year == test_year]

        entries = (yearly_data['Close'] > yearly_data['ZLMA']) & (yearly_data['Fisher'] > 0)
        exits = (yearly_data['Close'] < yearly_data['ZLMA']) & (yearly_data['Fisher'] < 0)

        portfolio = vbt.Portfolio.from_signals(
            close=yearly_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)

In [ ]:
# Define the stock symbol and time period
symbol = 'HWM'
start_date = '2016-11-01'
end_date = '2025-01-01'

# Download the stock data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']

# Perform walk-forward optimization
results = walk_forward_optimization_zlma_fisher(df, 2018, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']

    # Extend the data range to include the previous year for indicator calculation
    yearly_data = df[(df.index.year >= year - 1) & (df.index.year <= year)]

    # Apply ZLMA and Fisher Transform indicators
    yearly_data['ZLMA'] = calculate_zlma(yearly_data['Close'], params[0])
    yearly_data['Fisher'], yearly_data['Fisher_Signal'] = calculate_fisher_transform(yearly_data, params[1])

    # Keep only the second year to avoid missing values from indicator calculation
    yearly_data = yearly_data[yearly_data.index.year == year]

    # Define entry/exit conditions
    entries = (yearly_data['Close'] > yearly_data['ZLMA']) & (yearly_data['Fisher'] > 0)
    exits = (yearly_data['Close'] < yearly_data['ZLMA']) & (yearly_data['Fisher'] < 0)

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

In [ ]:
# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()